In [ ]:
sc

In [ ]:
fileLocation = "s3://cpsc5330s21/data-input/flight-data/2015-summary.csv"
flightData2015 = spark.read.option("inferScheme", "true").option("header", "true").csv(fileLocation)

In [ ]:
type(flightData2015)

In [ ]:
flightData2015.head(5)

In [ ]:
flightData2015.columns

In [ ]:
type(flightData2015.head(5))

### Schemas

In [ ]:
flightData2015.schema

In [ ]:
flightData2015.printSchema()

In [ ]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, FloatType, IntegerType

flightDataSchema = StructType([StructField('dest_country_name', StringType(), nullable=False),
                             StructField('origin_country_name', StringType(), nullable=False),
                             StructField('cnt', LongType(), nullable=False)])

In [ ]:
#flightData2015 = spark.read.option("inferScheme", "true").option("header", "true").csv(fileLocation)

In [ ]:
df = spark.read.format("csv").schema(flightDataSchema).option("header", "true").load(fileLocation)
df.cache()

In [ ]:
df.schema

In [ ]:
df.columns

In [ ]:
df.count()

In [ ]:
type(df.show(5))

In [ ]:
df.take(1)

In [ ]:
print(df.take(1)[0]['dest_country_name'])
print(df.take(1)[0][0])


### Other Ways to Create Data Frames

In [ ]:
# From a list of rows -- also, nested schema
from pyspark.sql.types import Row
nameSchema = StructType([StructField('first', StringType()),
                         StructField('last', StringType())])

nameIDSchema = StructType([StructField('name', nameSchema), StructField('age', LongType())])

r1 = Row(('manny', 'pep'), 10)
r2 = Row(('moe', 'pep'), 20)

nameIDdf = spark.createDataFrame([r1, r2], nameIDSchema)
nameIDdf.show(2)
nameIDdf.select('name.first').show()

In [ ]:
# From an RDD
from pyspark.sql import SQLContext
textcorporaLocation = 's3://cpsc5330s21/data-input/textcorpora/'
f = sc.wholeTextFiles(textcorporaLocation) 
sq = SQLContext(sc)
documents = sq.createDataFrame(f, StructType([StructField('filename', StringType()), StructField('document', StringType())]))
print(documents.count())
documents.show(1)

### Columns

In [ ]:
# From Pandas we would expect 
print(df.cnt)
print(df['cnt'])

In [ ]:
from pyspark.sql.functions import col
print(col('count'))
print(type(col('count')))

In [ ]:
# Columns can be used to form expressions
from pyspark.sql.functions import sum

print(type(col('cnt') + 5))
print(type(sum('cnt')))
print(type(col('dest_country_name') > col('origin_country_name')))

In [ ]:
from pyspark.sql.functions import sum
print(df.select(sum(col('cnt'))))
s = df.select(sum(col('cnt')))

In [ ]:
s.count()

In [ ]:
s.take(1)

In [ ]:
df.select(sum('cnt')).collect()

In [ ]:
df.select(sum('cnt')).collect()[0][0]

In [ ]:
df.select(sum(col('cnt') * 3)).collect()[0]

In [ ]:
print(df.select(sum(col('count') * 3)).collect()[0]['sum((count * 3))'])
print(df.select(sum(col('count') * 3)).collect()[0][0])

### Select 

In [ ]:
df.columns

In [ ]:
from pyspark.sql.functions import expr
df.select(expr("dest_country_name AS dcn")).show()

In [ ]:
df.select(expr("*"), expr("dest_country_name = origin_country_name as withinCountry")).show()

In [ ]:
df.select("dest_country_name").withColumnRenamed("dest_country_name", "dcn").show()

In [ ]:
df.drop("count").show()

### SQL Interface

In [ ]:
df.createOrReplaceTempView("dftable")

In [ ]:
spark.sql("""SELECT DEST_COUNTRY_NAME, count(*) FROM dftable GROUP BY DEST_COUNTRY_NAME ORDER BY count(*) desc""").show()

In [ ]:
dataFrameWay =   df.groupBy("DEST_COUNTRY_NAME").count()
sqlWay       =   spark.sql("""SELECT DEST_COUNTRY_NAME, count(*) FROM dftable GROUP BY DEST_COUNTRY_NAME""")

In [ ]:
type(sqlWay)

In [ ]:
dataFrameWay.explain()

In [ ]:
sqlWay.explain()

### Filtering Rows

In [ ]:
df.filter(col("cnt") > 1).show(5)

In [ ]:
df.select("dest_country_name", "origin_country_name").\
    filter(col("cnt") > 1).\
    filter(col("dest_country_name") != "United States").\
    show(5)

In [ ]:
chain = df.select("dest_country_name", "origin_country_name").filter(col("cnt") > 1).filter(col("dest_country_name") != "United States")
sql = spark.sql("""SELECT dest_country_name, origin_country_name from dftable where cnt > 1 and dest_country_name != 'United States'""")

In [ ]:
chain.explain()
sql.explain()

### Sorting

In [ ]:
df.orderBy(col("dest_country_name").asc(), col("cnt").desc()).show(5)

### UDFs and Broadcast Variables

In [ ]:
from pyspark.sql.functions import udf, col, column

def shorten(country):
    if len(country) < 3:
        return country
    else:
        return country[0:3]

shortenUDF = udf(lambda c: shorten(c), StringType())

In [ ]:
df.withColumn("shortdest", shortenUDF(col("dest_country_name"))).drop('dest_country_name').show()

In [ ]:
countryAbbrevs = {'Algeria': 'ALG', 
                  'Angola': 'ANGO', 
                  'Anguilla': 'ANGU', 
                  'Antigua and Barbuda': 'AAB', 
                  'United States': 'US'}
caBroadcast = sc.broadcast(countryAbbrevs)


In [ ]:
print(caBroadcast)
print(caBroadcast.value)

In [ ]:
def abbreviate(country):
    return caBroadcast.value.get(country, country)
 
abbreviateUDF = udf(lambda c: abbreviate(c), StringType())

In [ ]:
df.withColumn("abbrevdest", abbreviateUDF(col("dest_country_name"))).show()

### Grouping and Aggregating

In [ ]:
df.columns

In [ ]:
df.select('origin_country_name', 'cnt').groupBy('origin_country_name').count().show()

In [ ]:
df.select('origin_country_name', 'cnt').groupBy('origin_country_name').sum().orderBy(col("sum(cnt)").desc()).show()

In [ ]:
from pyspark.sql.functions import collect_set
cset = df.select('origin_country_name', 'dest_country_name').\
        groupBy(col('origin_country_name')).\
        agg(collect_set('dest_country_name'))
cset.show()

In [ ]:
cset.schema

In [ ]:
cset.filter(col('origin_country_name') == 'United States').show()


In [ ]:
len(cset.filter(col('origin_country_name') == 'United States').collect()[0][1])

In [ ]:
dfgrouped = df.select('origin_country_name', 'dest_country_name').\
    groupBy(col('origin_country_name')).\
    agg(collect_set('dest_country_name')).\
    withColumnRenamed('collect_set(dest_country_name)', 'dests').\
    filter(col('origin_country_name') == 'United States')
dfgrouped.show()

In [ ]:
from pyspark.sql.functions import explode
exploded = dfgrouped.withColumn('exploded', explode(col('dests'))).drop('dests')

In [ ]:
exploded.show()

### Join

In [ ]:
tfidfFileLocation = 's3://cpsc5330s21/data-input/tfidf/tfidf.tsv'
tfidfSchema = StructType([StructField('docid', StringType()),
                          StructField('term', StringType()),
                          StructField('tfidf', FloatType())])

tfidf = spark.read.format("csv").schema(tfidfSchema).option('header', False).option('delimiter', "\t").load(tfidfFileLocation)

In [ ]:
tfidf.show(10)

In [ ]:
dfSchema = StructType([StructField('term', StringType()),
                       StructField('df', IntegerType())])

dfData = [Row("freak", 2), Row("free", 10), Row("freed", 3), Row("freedom", 5), Row("freeze", 1)]
docFreq = spark.createDataFrame(dfData, dfSchema)
docFreq.show()

In [ ]:
j = tfidf.join(docFreq, tfidf["term"] == docFreq["term"])

In [ ]:
j.count()

In [ ]:
j.show(20)

In [ ]:
df2 = docFreq.withColumnRenamed("term", "term2")
j2 = tfidf.join(df2, tfidf["term"] == df2["term2"])

In [ ]:
j2.columns

In [ ]:
j3 = j2.drop("term2")

In [ ]:
j3.show(10)

In [ ]:
j4 = tfidf.join(docFreq, "term")

In [ ]:
j4.show(3)